In [4]:
import sys

sys.path.append("/home/daniel/Projects/3dVertPois")
sys.path.append("/home/daniel/Projects/3dVertPois/src")
import pandas as pd
from src.eval import compute_overall_metrics, create_prediction_df

In [5]:
dm_path = "/home/daniel/Projects/gruber/surface/all_pois/freeze/SA-DenseNet-PatchTransformer/version_4/data_module_params.json"
model_path = "/home/daniel/Projects/gruber/surface/all_pois/freeze/SA-DenseNet-PatchTransformer/version_4/checkpoints/sad-pt-epoch=44-fine_mean_distance_val=2.33.ckpt"
val_df = create_prediction_df(dm_path, model_path, joint=False, split="val")
test_df = create_prediction_df(
    dm_path, model_path, joint=False, split="test", alternative_poi_ending="poi.json"
)

In [6]:
# Drop the rows where loss_mask is false
val_df = val_df[val_df.loss_mask]
test_df = test_df[test_df.loss_mask]
test_df.head()

,subject,vertebra,poi_idx,target,coarse,refined,coarse_proj,refined_proj,coarse_proj_dist,refined_proj_dist,loss_mask,coarse_error,refined_error,coarse_proj_error,refined_proj_error
0,WS-06,3,81,"[65, 41, 48]","[65.04377, 42.669796, 48.697636]","[64.5924, 42.080364, 48.96319]","[65, 43, 48]","[64, 42, 49]",0.773076,0.598958,True,1.810202,1.503681,2.000000,1.732051
1,WS-06,3,101,"[64, 87, 50]","[64.157906, 87.145325, 48.382347]","[65.25885, 86.57883, 48.682854]","[64, 87, 48]","[65, 87, 49]",0.438455,0.587343,True,1.631825,1.870017,2.000000,1.414214
2,WS-06,3,102,"[64, 73, 54]","[64.44524, 73.65172, 52.8178]","[64.73712, 73.104744, 53.446117]","[64, 73, 53]","[65, 73, 53]",0.810043,0.528296,True,1.421468,0.927958,1.000000,1.414214
3,WS-06,3,103,"[65, 87, 36]","[65.28894, 85.97514, 36.932915]","[65.09692, 86.52205, 36.314697]","[65, 86, 36]","[65, 87, 36]",0.976952,0.580400,True,1.415679,0.580400,1.000000,0.000000
4,WS-06,3,104,"[64, 72, 42]","[64.9566, 71.29375, 42.622856]","[64.23651, 71.65941, 42.71393]","[65, 72, 43]","[64, 72, 43]",0.801820,0.503763,True,1.342319,0.825611,1.414214,1.000000


In [37]:
# Keep only the POIs in the sagittal plane i.e. poi_idx 101,102,103,104,125,127,81,134,136
val_df = val_df[val_df.poi_idx.isin([101, 102, 103, 104, 125, 127, 81, 134, 136])]
test_df = test_df[test_df.poi_idx.isin([101, 102, 103, 104, 125, 127, 81, 134, 136])]
test_df.head()

,subject,vertebra,poi_idx,target,coarse,refined,coarse_proj,refined_proj,coarse_proj_dist,refined_proj_dist,loss_mask,coarse_error,refined_error,coarse_proj_error,refined_proj_error
0,WS-06,3,81,"[65, 41, 48]","[64.16144, 44.324917, 48.389175]","[64.404655, 43.440525, 48.722305]","[64, 44, 48]","[64, 43, 48]",0.532063,0.937834,True,3.451046,2.613871,3.162278,2.236068
1,WS-06,3,101,"[64, 87, 50]","[64.56482, 86.707504, 48.80835]","[64.38295, 87.11289, 49.289684]","[65, 87, 49]","[64, 87, 49]",0.558270,0.493267,True,1.350780,0.814827,1.414214,1.000000
2,WS-06,3,102,"[64, 73, 54]","[64.43835, 73.37381, 54.16559]","[64.16742, 73.58826, 54.004227]","[64, 73, 54]","[64, 74, 54]",0.599417,0.444499,True,0.599417,0.611632,0.000000,1.000000
3,WS-06,3,103,"[65, 87, 36]","[64.79538, 85.71253, 36.002552]","[64.86791, 86.28996, 35.959534]","[65, 86, 36]","[65, 86, 36]",0.352865,0.321190,True,1.303629,0.723352,1.000000,1.000000
4,WS-06,3,104,"[64, 72, 42]","[64.843956, 72.193985, 41.78711]","[64.27244, 71.78385, 42.22177]","[65, 72, 42]","[64, 72, 42]",0.327570,0.412462,True,0.891748,0.412462,1.000000,0.000000


In [38]:
# Add a column split to the dataframes then concatenate them
val_metrics = compute_overall_metrics(val_df)
val_metrics["Split"] = "Val"
test_metrics = compute_overall_metrics(test_df)
test_metrics["Split"] = "Test"

metrics = pd.concat([val_metrics, test_metrics])
print(metrics)

                    Mean Error  Median Error       MSE  Accuracy  Max Error  \
coarse_error          2.056038      1.906737  5.229245  0.544855   7.388394   
refined_error         1.680683      1.413865  3.907427  0.711082   8.743143   
coarse_proj_error     1.987036      2.000000  5.229551  0.496042   7.681146   
refined_proj_error    1.677755      1.414214  4.209763  0.629288   8.717798   
coarse_error          2.050937      1.861986  5.343517  0.548077   8.226601   
refined_error         1.765032      1.548714  4.319936  0.688187   9.358090   
coarse_proj_error     1.960866      1.732051  5.281593  0.521978   8.544004   
refined_proj_error    1.772555      1.414214  4.552198  0.589286   9.899495   

                   Split  
coarse_error         Val  
refined_error        Val  
coarse_proj_error    Val  
refined_proj_error   Val  
coarse_error        Test  
refined_error       Test  
coarse_proj_error   Test  
refined_proj_error  Test  


In [39]:
# For pretty printing, bring split column to the front
cols = metrics.columns.tolist()
cols = cols[-1:] + cols[:-1]
metrics = metrics[cols]

# Make the index column a regular column called 'Prediction Type'
metrics.reset_index(inplace=True)
metrics.rename(columns={"index": "Prediction Type"}, inplace=True)
# Rename the entries in the 'Prediction Type' column coarse_error -> Coarse, refined_error -> Fine, coarse_proj_error -> Coarse with projection, refined_proj_error -> Fine with projection
metrics["Prediction Type"] = metrics["Prediction Type"].replace(
    {
        "coarse_error": "Coarse",
        "refined_error": "Fine",
        "coarse_proj_error": "Coarse with projection",
        "refined_proj_error": "Fine with projection",
    }
)

# Reorder the columns for pretty printing
metrics = metrics[
    [
        "Split",
        "Prediction Type",
        "Mean Error",
        "Median Error",
        "MSE",
        "Accuracy",
        "Max Error",
    ]
]
# drop the max error
metrics = metrics.drop(columns=["Max Error"])
print(metrics.to_latex(index=False, float_format="%.2f"))

,Split,Mean Error,Median Error,MSE,Accuracy,Max Error
coarse_error,Val,2.056038,1.906737,5.229245,0.544855,7.388394
refined_error,Val,1.680683,1.413865,3.907427,0.711082,8.743143
coarse_proj_error,Val,1.987036,2.000000,5.229551,0.496042,7.681146
refined_proj_error,Val,1.677755,1.414214,4.209763,0.629288,8.717798
coarse_error,Test,2.050937,1.861986,5.343517,0.548077,8.226601
refined_error,Test,1.765032,1.548714,4.319936,0.688187,9.358090
coarse_proj_error,Test,1.960866,1.732051,5.281593,0.521978,8.544004
refined_proj_error,Test,1.772555,1.414214,4.552198,0.589286,9.899495


In [16]:
metrics

,Split,Prediction Type,Mean Error,Median Error,MSE,Accuracy,Max Error
0,Val,Coarse,2.605554,2.235278,9.698483,0.426188,14.241716
1,Val,Fine,2.400178,1.977584,8.724401,0.506572,14.286979
2,Val,Coarse with projection,2.562638,2.168455,9.375610,0.446916,15.022503
3,Val,Fine with projection,2.495859,2.065476,9.165537,0.479272,13.579013
4,Test,Coarse,2.836674,2.358810,12.520220,0.394475,29.328714
5,Test,Fine,2.658057,2.141098,11.588312,0.459692,30.915007
6,Test,Coarse with projection,2.836641,2.343405,12.438401,0.391304,29.140836
7,Test,Fine with projection,2.762990,2.270329,12.070157,0.414855,30.503054
